<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align = "center"> Spark Fundamentals I - Introduction to Spark</h1>
<h2 align = "center"> Scala - Working with Scala Libraries</h2>
<br align = "left">

**Related free online courses:**

Related courses can be found in the following learning paths:

- [Spark Fundamentals path](http://cocl.us/Spark_Fundamentals_Path)
- [Big Data Fundamentals path](http://cocl.us/Big_Data_Fundamentals_Path)

<img src="http://spark.apache.org/images/spark-logo.png" height=100>

## Creating a Spark application using Spark SQL

Spark SQL provides the ability to write relational queries to be run on Spark. There is the abstraction SchemaRDD which is to create an RDD in which you can run SQL, HiveQL, and Scala. In this lab section, you will use SQL to find out the average weather and precipitation for a given time period in New York. The purpose is to demonstrate how to use the Spark SQL libraries on Spark.

### Please note that in Spark 1.3 DataFrames have replaced schemaRDDs however, it is still possible to switch between the two for supporting legacy systems. DataFrames is the recommended method going forward

### Let's first download the data that we will be working with in this lab

In [ ]:
// download module to run shell commands within this notebook
import sys.process._

If you completed the **Getting Started** lab or the **RDD** lab, then you should have the data downloaded and unzipped in the */resources/jupyterlab/labs/BD0211EN/LabData/* directory. Otherwise, please uncomment **the last two lines of code** in each of the following cells to download and unzip the data.

In [ ]:
// download data from IBM Servier
// this may take ~30 seconds depending on your internet speed

//"wget --quiet https://cocl.us/BD0211EN_Data" !

//println("Data Downloaded!")

In [ ]:
// unzip the folder's content into "resources" directory

//"unzip -q -o -d /resources/jupyterlab/labs/BD0211EN/ BD0211EN_Data" !

//println("Data Extracted!")

The data is in a folder called **LabData**. Let's list all the files in the data that we just downloaded and extracted.

In [ ]:
// list the extracted files
"ls -1 /resources/jupyterlab/labs/BD0211EN/LabData/" !

Let's take a look at the nycweather data. So run the following code:

In [ ]:
val lines = scala.io.Source.fromFile("/resources/jupyterlab/labs/BD0211EN/LabData/nycweather.csv").mkString

println(lines)

There are three columns in the dataset, the date, the mean temperature in Celsius, and the precipitation for the day. Since we already know the schema, we will infer the schema using reflection.

You will first need to define the SparkSQL context. Do so by creating it from an existing SparkContext. Type in:

In [ ]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

Next, you need to import a library for creating a SchemaRDD. Type this:

In [ ]:
import sqlContext.implicits._

Create a case class in Scala that defines the schema of the table. Type in:

In [ ]:
case class Weather(date: String, temp: Int, precipitation: Double)

Create the RDD of the Weather object:

In [ ]:
val weather = sc.textFile("/resources/jupyterlab/labs/BD0211EN/LabData/nycweather.csv").map(_.split(",")). map(w => Weather(w(0), w(1).trim.toInt, w(2).trim.toDouble)).toDF()

You first load in the file, and then you map it by splitting it up by the commas and then another mapping to get it into the Weather class.

Next you need to register the RDD as a table. Type in:

In [ ]:
weather.registerTempTable("weather")

At this point, you are ready to create and run some queries on the RDD. You want to get a list of the hottest dates with some precipitation. Type in:

In [ ]:
val hottest_with_precip = sqlContext.sql("SELECT * FROM weather WHERE precipitation > 0.0 ORDER BY temp DESC")

hottest_with_precip.collect()

## Creating a Spark application using MLlib

In this section, Spark will be used to acquire the K-Means clustering for drop-off latitudes and longitudes of taxis for 3 clusters. The sample data contains a subset of taxi trips with hack license, medallion, pickup date/time, drop off date/time, pickup/drop off latitude/longitude, passenger count, trip distance, trip time and other information. As such, this may give a good indication of where to best to hail a cab.

Remember, this is only a subset of the file that you used in a previous exercise. If you ran this exercise on the full dataset, it would take a long time as we are only running on a test environment with limited resources.

Import the needed packages for K-Means algorithm and Vector packages:

In [ ]:
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.mllib.linalg.Vectors

Create an RDD

In [ ]:
val taxiFile = sc.textFile("/resources/jupyterlab/labs/BD0211EN/LabData/nyctaxisub.csv")

Determine the number of rows in taxiFile.

In [ ]:
taxiFile.count()

Cleanse the data.

In [ ]:
val taxiData=taxiFile.filter(_.contains("2013")).
    filter(_.split(",")(3)!="" ).    //dropoff_latitude
    filter(_.split(",")(4)!="")      //dropoff_longitude

The first filter limits the rows to those that occurred in the year 2013. This will also remove any header in the file. The third and fourth columns contain the drop off latitude and longitude. The transformation will throw exceptions if these values are empty.

Do another count to see what was removed.

In [ ]:
taxiData.count()

In this case, if we had used the full set of data, it would have filtered out a great many more lines.

To fence the area roughly to New York City use this command:

In [ ]:
val taxiFence=taxiData.
    filter(_.split(",")(3).toDouble>40.70).
    filter(_.split(",")(3).toDouble<40.86).
    filter(_.split(",")(4).toDouble>(-74.02)).
    filter(_.split(",")(4).toDouble<(-73.93))

Determine how many are left in taxiFence:

In [ ]:
taxiFence.count()

Approximately, 43,354 rows were dropped since these drop-off points are outside of New York City.

Create Vectors with the latitudes and longitudes that will be used as input to the K-Means algorithm.

In [ ]:
val taxi=taxiFence.
    map{
        line=>Vectors.dense(
            line.split(',').slice(3,5).map(_ .toDouble)
        )
    }

In [ ]:
val iterationCount=10
val clusterCount=3

val model=KMeans.train(taxi,clusterCount,iterationCount)
val clusterCenters=model.clusterCenters.map(_.toArray)

clusterCenters.foreach(lines=>println(lines(0),lines(1)))

Now we know the map co-ordinates. Not surprisingly, the second point is between the Theater District and Grand Central. The third point is in The Village, NYU, Soho and Little Italy area. The first point is the Upper East Side, presumably where people are more likely to take cabs than subways.



## Creating a Spark application using Spark Streaming

This section focuses on Spark Streams, an easy to build, scalable, stateful (e.g. sliding windows) stream processing library. Streaming jobs are written the same way Spark batch jobs are coded and support Java, Scala and Python. In this exercise, taxi trip data will be streamed using a socket connection and then analyzed to provide a summary of number of passengers by taxi vendor. This will be implemented in the Spark shell using Scala.

There are two relevant files for this section. The first one is the nyctaxi100.csv which will serve as the source of the stream. The other file is a python file, taxistreams.py, which will feed the csv file through a socket connection to simulate a stream.

### <span style="color: red">IN ORDER TO START THE STREAM PLEASE OPEN A NEW PYTHON NOTEBOOK AND RUN THE CODE BELOW IN IT:</span> 

To open a new Python notebook click on the + icon on the top left corner of this page, above the line displaying the directory where this Notebook resides. Choose PYTHON 3 and then copy and past the code below into the cell in the new JupyterLab Notebook. Run the cell as normal. To interrupt the kernel hit the STOP button in the Action buttons above.

```
!python /resources/jupyter/labs/BD0211EN/LabData/taxistreams.py

```

Once started, the program will bind and listen to the localhost socket 7777. When a connection is made, it will read ‘nyctaxi100.csv’ and send across the socket. The sleep is set such that one line will be sent every 0.5 seconds, or 2 rows a second. This was intentionally set to a high value to make it easier to view the data during execution.

Turn off logging so that you can see the output of the application and Import the required libraries:

In [ ]:
import org.apache.log4j.Logger
import org.apache.log4j.Level
Logger.getLogger("org").setLevel(Level.OFF)
Logger.getLogger("akka").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.streaming._
import org.apache.spark.streaming.StreamingContext._

Create the StreamingContext by using the existing SparkContext (sc). It will be using a 1 second batch interval, which means the stream is divided to 1 second batches and each batch becomes a RDD. This is intentional to make it easier to read the data during execution.

In [ ]:
val ssc = new StreamingContext(sc, Seconds(1))

Create the socket stream that connects to the localhost socket 7777. This matches the port that the Python script is listening on. Each batch from the Stream be a lines RDD.

In [ ]:
val lines = ssc.socketTextStream("localhost", 7777)

Next, put in the business logic to split up the lines on each comma and mapping pass(15), which is the vendor, and pass(7), which is the passenger count. Then this is reduced by key resulting in a summary of number of passengers by vendor.

In [ ]:
val pass = lines.map(_.split(",")).
    map(pass=>(pass(15), pass(7).toInt)).
    reduceByKey(_+_)

Print out to the console:

In [ ]:
pass.print()

The next two line starts the stream. 

In [ ]:
ssc.start()
ssc.awaitTermination()

It will take a few cycles for the connection to be recognized, and then the data is sent. In this case, 2 rows per second of taxi trip data is receive in a 1 second batch interval.

In the Python terminal, the contents of the file are printed as they are streamed.

**Note: TO STOP THE STREAM PLEASE INTERRUPT THE KERNEL IN BOTH THE OTHER PYTHON NOTEBOOK AND THIS NOTEBOOK. THEN RESTART THIS NOTEBOOK'S KERNEL TO CONTINUE ONTO THE GRAPHX APPLICATION**

This is just a simple example showing how you can take streaming data into Spark and do some type of processing on it. In the case here, the taxi and the number of passengers was extracted from the data stream.

## Creating a Spark application using GraphX

Users.txt is a set of users and followers is the relationship between the users. Take a look at the contents of these two files.

In [ ]:
println("Users: ")
println(scala.io.Source.fromFile("/resources/jupyterlab/labs/BD0211EN/LabData/users.txt").mkString)

println("Followers: ")
println(scala.io.Source.fromFile("/resources/jupyterlab/labs/BD0211EN/LabData/followers.txt").mkString)

Import the GraphX package:

In [ ]:
import org.apache.spark.graphx._

Create the users RDD and parse into tuples of user id and attribute list:

In [ ]:
val users = (sc.textFile("/resources/jupyterlab/labs/BD0211EN/LabData/users.txt").map(line => line.split(",")).map(parts => (parts.head.toLong, parts.tail)))

users.take(5).foreach(println)

Parse the edge data, which is already in userId -> userId format

In [ ]:
val followerGraph = GraphLoader.edgeListFile(sc, "/resources/jupyterlab/labs/BD0211EN/LabData/followers.txt")

Attach the user attributes

In [ ]:
val graph = followerGraph.outerJoinVertices(users) {
    case (uid, deg, Some(attrList)) => attrList
    case (uid, deg, None) => Array.empty[String]
}

Restrict the graph to users with usernames and names:

In [ ]:
val subgraph = graph.subgraph(vpred = (vid, attr) => attr.size == 2)

Compute the PageRank

In [ ]:
val pagerankGraph = subgraph.pageRank(0.001)

Get the attributes of the top pagerank users

In [ ]:
val userInfoWithPageRank = subgraph.outerJoinVertices(pagerankGraph.vertices) {
    case (uid, attrList, Some(pr)) => (pr, attrList.toList)
    case (uid, attrList, None) => (0.0, attrList.toList)
}

Print the line out:

In [ ]:
println(userInfoWithPageRank.vertices.top(5)(Ordering.by(_._2._1)).mkString("\n"))

<div class="alert alert-success alertsuccess" style="margin-top: 20px">
    <strong>Tip</strong>: Enjoyed using Jupyter notebooks with Spark? Get yourself a free 
    <a href="http://cocl.us/DSX_on_Cloud">IBM Cloud</a> account where you can use Data Science Experience notebooks
    and have <em>two</em> Spark executors for free!
</div>

## Summary

Having completed this exercise, you should have some familiarity with using the Spark libraries. In particular, you use Spark SQL to effectively query data inside of Spark. You used Spark Streaming to process incoming streams of batch data. You used Spark's MLlib to compute the *k*-means algorithm to find the best place to hail a cab. Finally, you used Spark's GraphX library to perform and parallel graph calculations on a dataset to find the attributes of the top users.

This notebook is part of the free course on **cognitiveclass.ai** called *Spark Fundamentals I*. If you accessed this notebook outside the course, you can take this free self-paced course, online by going to: http://cocl.us/Spark_Fundamentals_I

### About the Authors:  
Hi! It's Alex Aklson, one of the authors of this notebook. I hope you found this lab educational! There is much more to learn about Spark but you are well on your way. Feel free to connect with me if you have any questions.
<hr>